# TensorFlowによるニューラルネットワーク

TensorFlowでニューラルネットワークのプログラムを実装する方法として，セッションを準備して実行する方法，Estimatorを利用する方法，Eagerを利用する方法があります．ここでは，まずセッションを準備する方法について説明します．

必要なパッケージをインポートします．
また，GPUが利用可能かを確認します．

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.test.gpu_device_name()

'/device:GPU:0'

MNISTデータセットをダウンロードします．以下を実行するとWARNINGがいくつか表示されますが，問題ありません．

In [2]:
mnist = input_data.read_data_sets("./", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


パラメータ設定を行います．学習率(learning_rate)，学習更新回数(num_steps)，ミニバッチサイズ(batch_size)，画面出力するステップ数(display_step)を用意します．
また，ニューラルネットワークのパラメータとして，中間層のユニット数(n_hidden_1とn_hidden_2)を256とします．入力層のユニット数(num_input)は784，出力層のユニット数は(num_classes)は10とします．

In [ ]:
learning_rate = 0.1
num_steps = 5000
batch_size = 128
display_step = 100


n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784 
num_classes = 10



学習データと教師ラベルを準備します．
placeholderは，データを格納する入れ物です．ここでは，入れ物のサイズだけを決めて，具体的な値は実行する時に与えます．

In [ ]:
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

ニューラルネットワークのパラメータを記憶しておく重み(weights)とバイアス(biases)を定義します．入力層と中間層の間の重みをh1，１つ目の中間層と2つ目の中間層の間の重みをh2，2つ目の中間層と出力層の間の重みをoutとします．バイアスも同様それぞれb1, b2, outとします．

In [ ]:
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

ネットワークモデルをneural_netとして定義します．１層目(layer_1)は入力xと重みh1をmatmulで乗算し，addでバイアスb1を加算します．２層目(layer_2)は，１層目の出力layer_1と重みh2をmatmulで乗算し，addでバイアスb2を加算します．そして，２層目の出力layer_2と出力層の重みoutを乗算し，バイアスoutを加算します．その結果(out_layer)を戻り値とします．

In [ ]:
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

先ほど定義したネットワークモデルをlogitsとします．

In [ ]:
logits = neural_net(X)

誤差関数と最適化の方法を定義します．ここでは，物体認識を行うのでソフトマックスクロスエントロピーを誤差関数とします．最適化の方法には，Adamを利用します．誤差はminimizeで最小化するようにします．

In [ ]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

次に評価を行う準備をします．ネットワークからの出力値が最大となるクラスの番号と教師データが一致するかをequalで判定します．全てのデータに対する結果をまとめてaccuracyを算出します．

In [ ]:
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

ネットワークのパラメータはVariablesという値の変化を記憶するクラスで準備しています．最初にこれらの値を初期化します．

In [ ]:
init = tf.global_variables_initializer()

学習を実行するために，セッションをtf.Session()で準備します．そして，セッションをsess.runで実行しますが，最初に引数にinitを与えて初期化されたパラメータを渡します．for文内で繰り返し学習を行います．ここでは，まずミニバッチサイズ分の学習データと教師データを取得し，sess.run(XXX)で引数に指定したtrain_opを実行します．fee_dictとして与えたデータをネットワークに順伝播し，誤差を算出します．もし，更新回数がdisplay_stepで割り切れる場合は誤差と精度を画面に出力します．

In [29]:
with tf.Session() as sess:
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Test Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 9745.2559, Training Accuracy= 0.312
Step 100, Minibatch Loss= 497.6499, Training Accuracy= 0.867
Step 200, Minibatch Loss= 220.8164, Training Accuracy= 0.828
Step 300, Minibatch Loss= 86.5161, Training Accuracy= 0.898
Step 400, Minibatch Loss= 43.7917, Training Accuracy= 0.883
Step 500, Minibatch Loss= 70.9563, Training Accuracy= 0.859
Step 600, Minibatch Loss= 32.7685, Training Accuracy= 0.891
Step 700, Minibatch Loss= 36.7726, Training Accuracy= 0.883
Step 800, Minibatch Loss= 57.4573, Training Accuracy= 0.859
Step 900, Minibatch Loss= 38.2886, Training Accuracy= 0.898
Step 1000, Minibatch Loss= 40.9444, Training Accuracy= 0.867
Step 1100, Minibatch Loss= 46.4080, Training Accuracy= 0.867
Step 1200, Minibatch Loss= 32.3937, Training Accuracy= 0.867
Step 1300, Minibatch Loss= 29.7861, Training Accuracy= 0.883
Step 1400, Minibatch Loss= 50.4438, Training Accuracy= 0.859
Step 1500, Minibatch Loss= 22.7806, Training Accuracy= 0.883
Step 1600, Minibatch Loss= 65.46

##課題

1. 学習更新回数を1000，2000，5000と増やした場合の精度を比較しましょう

2. 中間層のユニット数を256から512または1024と増やした場合の精度を比較しましょう

3. 中間層の層数を2層から3層に増やしてみましょう
